# Descripción del proyecto

Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

- Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
- Crear un modelo para predecir el volumen de reservas en pozos nuevos.
- Elegir los pozos petrolíferos que tienen los valores estimados más altos.
- Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.



# Introducción

Este proyecto se enfoca en la identificación de las mejores ubicaciones para abrir 200 nuevos pozos de petróleo para OilyGiant. Para llevar a cabo esta tarea, se comenzará analizando datos sobre la calidad de crudo y el volumen de reservas de pozos existentes en tres regiones. A partir de estos datos, se desarrollará un modelo predictivo que estime el volumen de reservas en los nuevos pozos.

Una vez que se obtengan las estimaciones, se seleccionarán aquellos pozos con los valores más altos y se evaluará cuál de las regiones ofrece el mayor beneficio total. Además, se implementará la técnica de bootstrapping para analizar de manera rigurosa los beneficios y riesgos potenciales asociados con la apertura de los nuevos pozos. Este enfoque integral permitirá tomar decisiones estratégicas y fundamentadas en la optimización de la extracción de petróleo.


# Inicialización

In [74]:
# Cargamos todas las librerias que creemos que vamos a utilizar

import pandas as pd
import numpy as np
from numpy.random import RandomState
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

## Preparación de los datos

In [75]:
# cargamos los archivos en dataframes

df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [76]:
#validamos las dimensiones del dataframe df_0 (usamos shape) e imprimimos las primeras filas (usamos head)

print(df_0.shape)
print(df_0.head())
print()

#mostramos la informacion del dataframe con el metodo info
df_0.info()

(100000, 5)
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [77]:
#validamos las dimensiones del dataframe df_1 (usamos shape) e imprimimos las primeras filas (usamos head)

print(df_1.shape)
print(df_1.head())
print()

#mostramos la informacion del dataframe con el metodo info
df_1.info()

(100000, 5)
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [78]:
#validamos las dimensiones del dataframe df_2 (usamos shape) e imprimimos las primeras filas (usamos head)

print(df_2.shape)
print(df_2.head())
print()

#mostramos la informacion del dataframe con el metodo info
df_2.info()

(100000, 5)
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


### Duplicados y valores ausentes

In [79]:
# validamos si tenemos valores duplicados en el dataframe df_0 con el metodo duplicated
print(df_0.duplicated().sum())

0


In [80]:
# validamos si tenemos valores duplicados en el dataframe df_1 con el metodo duplicated
print(df_1.duplicated().sum())

0


In [81]:
# validamos si tenemos valores duplicados en el dataframe df_2 con el metodo duplicated
print(df_2.duplicated().sum())

0


In [82]:
#validamos si tenemos valores ausentes en el dataframe df_0 con el metodo isna
print(df_0.isna().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [83]:
#validamos si tenemos valores ausentes en el dataframe df_1 con el metodo isna
print(df_1.isna().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [84]:
#validamos si tenemos valores ausentes en el dataframe df_2 con el metodo isna
print(df_2.isna().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


### Resumen estadístico de las variables

In [85]:
# Mostramos un resumen estadístico de las variables de la tabla df_0 utilizando el metodo describe
print(df_0.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347


In [86]:
# Mostramos un resumen estadístico de las variables de la tabla df_1 utilizando el metodo describe
print(df_1.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%         1.153055      -4.813172       2.011479      57.085625
75%         8.621015      -1.332816       3.999904     107.813044
max        29.421755      18.734063       5.019721     137.945408


In [87]:
# Mostramos un resumen estadístico de las variables de la tabla df_2 utilizando el metodo describe
print(df_2.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288      -1.174820       0.130359      59.450441
50%         0.009424      -0.009482       2.484236      94.925613
75%         1.158535       1.163678       4.858794     130.595027
max         7.238262       7.844801      16.739402     190.029838


## Entrenamiento y prueba del modelo

### Division de los datos (región 0)

In [88]:
#separamos las caracteristicas y nuestro objetivo
features_0 = df_0.drop(['id', 'product'], axis=1)
target_0 = df_0['product']

In [89]:
#dividimos los datos en conjunto de entrenamiento y validacion 

# dividimos el entrenamiento (75%) y un conjunto de validación (25%)
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, test_size=0.25, random_state=12345)


### Entrenamos el modelo (región 0)

In [90]:
#entrenamos el modelo
model_0 = LinearRegression()

model_0.fit(features_train_0, target_train_0)

predicted_valid_0 = model_0.predict(features_valid_0) # Realizamos predicciones en el conjunto de validación


### Guardamos predicciones y respuestas correctas (región 0)

In [91]:
#Almacenamos las predicciones junto con las respuestas correctas en un DataFrame.

df_predictions_answers_0 = pd.DataFrame({'target_valid': target_valid_0, 'predicted_valid': predicted_valid_0})

print(df_predictions_answers_0)

       target_valid  predicted_valid
71751     10.038645        95.894952
80493    114.551489        77.572583
2655     132.603635        77.892640
53233    169.072125        90.175134
91141    122.325180        70.510088
...             ...              ...
12581    170.116726       103.037104
18456     93.632175        85.403255
73035    127.352259        61.509833
63834     99.782700       118.180397
43558    177.821022       118.169392

[25000 rows x 2 columns]


### Volumen medio de reservas (región 0)

In [92]:
# calculamos el volumen medio predicho con el metodo mean
mean_predicted_reserves_0 = predicted_valid_0.mean()
print(f'Volumen medio de reservas predicho (Región 0): {mean_predicted_reserves_0}')

# Calculamos el RMSE 
rmse_0 = mean_squared_error(target_valid_0, predicted_valid_0)**0.5
print(f'RMSE (Región 0): {rmse_0}')

Volumen medio de reservas predicho (Región 0): 92.59256778438035
RMSE (Región 0): 37.5794217150813


### Analisis de resultados (region 0)

El volumen medio de reservas predicho por el modelo es de 92.59 de miles de barriles. Esto podria indicar que en promedio, el modelo está prediciendo un volumen de reservas aproximado a este valor para los pozos en esta región.

mientras que el RMSE fue de 37.58 miles de barriles. Esto significa que en promedio, las predicciones del modelo tienen un error de aproximadamente 37.58 miles de barriles respecto al valor real.
Esto nos dice, que el error es relativamente bajo (pero no insignificante) en comparación con el volumen medio predicho (92.59 miles de barriles), lo que podria sugerir que el modelo es capaz de hacer predicciones razonablemente precisas.


### Región 1 y Región 2

In [93]:
# Automatizamos en una funcion los pasos anteriores para validar en las demas regiones

def train_and_evaluate_model(data, region_name):
    # Separamos las  características y el objetivo
    features = data.drop(['id', 'product'], axis=1)
    target = data['product']
    
    # Dividimos los datos en entrenamiento y validación
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    
    # Entrenamos el modelo
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    # Hacemos las predicciones
    predicted_valid = model.predict(features_valid)
    
    # Calculamos el volumen medio y el RMSE
    mean_predicted_reserves = predicted_valid.mean()
    rmse = mean_squared_error(target_valid, predicted_valid)**0.5
    
    print(f'Resultados de la región: {region_name}')
    print(f'Volumen medio de reservas predicho: {mean_predicted_reserves}')
    print(f'RMSE: {rmse}')
    return predicted_valid, target_valid

#  ejecutamos las siguientes variables para cada region
predictions_0, target_valid_0 = train_and_evaluate_model(df_0, "región 0")
print()
predictions_1, target_valid_1 = train_and_evaluate_model(df_1, "región 1")
print()
predictions_2, target_valid_2 = train_and_evaluate_model(df_2, "región 2")


Resultados de la región: región 0
Volumen medio de reservas predicho: 92.59256778438035
RMSE: 37.5794217150813

Resultados de la región: región 1
Volumen medio de reservas predicho: 68.728546895446
RMSE: 0.893099286775617

Resultados de la región: región 2
Volumen medio de reservas predicho: 94.96504596800489
RMSE: 40.02970873393434


vemos que la región 1 tiene el volumen medio más bajo, pero el modelo es extremadamente preciso en esta región. Con un RMSE de solo 0.893, lo que nos dice que las predicciones del modelo son muy cercanas a los valores reales, y que minimiza el riesgo de tomar decisiones incorrectas basandonos en las predicciones.

en la región 2 se tiene el volumen medio más alto de las tres regiones (94.96), lo cual es atractivo desde el punto de vista del rendimiento potencial. Sin embargo, el RMSE también es alto(40.02), lo que sugiere que el modelo podria tener dificultades para hacer predicciones precisas en esta región.

volviendo a la region 0 y teniendo en cuenta la informacion de la region 1, el valor de rmse de 37.58 ya no parece tan bajo, incluso se ve considerablemente alto como el de la region 2, pero manejable ya que no es extremadamente alto para una predicción en el contexto de la exploración de petróleo.

Si la prioridad es la precisión en las predicciones (para reducir el riesgo), la Región 1 seria la mejor opción, ya que el modelo es muy preciso allí.
Si el objetivo es maximizar el volumen de reservas, las Regiones 0 y 2 son mejores, pero se debe tener en cuenta el mayor riesgo debido a que es mas alto el RMSE.

## Cálculo de ganancias

### Almacenamiento de los valores para el cálculo

In [94]:
# almacenamos las variables que necesitamos 

budget = 100e6  # presupuesto de 100 millones de dólares
wells = 200  # cantidad de pozos
income_per_unit = 4500  # ingresos por unidad de producto en dolares
break_even_threshold = budget / wells # umbral de equilibrio para evitar perdidas(en dolares) 
units_break_even_threshold = break_even_threshold / income_per_unit # umbral de equilibrio para evitar perdidas(en miles de barriles)

print(f'Cada pozo debe producir al menos {break_even_threshold} dólares para no tener pérdidas,')
print(f'es decir, al menos {units_break_even_threshold:.2f} miles de barriles para no tener pérdidas.')


Cada pozo debe producir al menos 500000.0 dólares para no tener pérdidas,
es decir, al menos 111.11 miles de barriles para no tener pérdidas.


### Comparación con las reservas de cada región

In [95]:
# Calculamos la cantidad media de reservas por región

mean_reserves_0 = df_0['product'].mean()
mean_reserves_1 = df_1['product'].mean()
mean_reserves_2 = df_2['product'].mean()

print(f'Reservas promedio en la región 0: {mean_reserves_0:.2f} miles de barriles')
print(f'Reservas promedio en la región 1: {mean_reserves_1:.2f} miles de barriles')
print(f'Reservas promedio en la región 2: {mean_reserves_2:.2f} miles de barriles')


Reservas promedio en la región 0: 92.50 miles de barriles
Reservas promedio en la región 1: 68.83 miles de barriles
Reservas promedio en la región 2: 95.00 miles de barriles


### Conclusiones de reservas en cada región

Las reservas promedio de la region 0 y 2 con 92.5 y 95 miles de barriles respectivamente, todavía están por debajo del umbral de equilibrio (111.11). Esto indica que en promedio, los pozos de ambas regiones no alcanzarían el punto de equilibrio para evitar pérdidas.

Las reservas promedio en la Región 1 es aún más baja (68.83), lo que indica que los pozos de esta región tienen el mayor riesgo de pérdidas si solo se consideran las reservas promedio. Está significativamente por debajo del umbral de equilibrio.

de acuerdo a lo anterior, ninguna de las tres regiones alcanza el umbral de 111.11 miles de barriles necesario para evitar pérdidas en promedio. Esto indicaria que, en promedio, los pozos individuales en todas las regiones podrían generar pérdidas si no se hace un analisis mas a fondo.

En lugar de basarnos en el promedio general de cada región, se podria seleccionar los pozos con las mayores reservas y calcular el beneficio para estos. será necesario seleccionar los 200 pozos con las predicciones más altas en cada región y luego calcular los ingresos basados en estas predicciones. Esto nos ayuda a saber cual de las tres regiones tiene el mayor potencial de ganancia.




## Ganancias de los pozos con mayor valor

### eleccion de los 200 pozos con valores de prediccion mas altos en cada región

In [96]:
def top_200_wells(predictions, data_valid):
    # Combinamos las predicciones con los datos de validación
    wells_data = pd.DataFrame({
        'predictions': predictions,
        'actual': data_valid.reset_index(drop=True)  # Reservas reales
    })

    # Ordenamos por las predicciones y seleccionamos los 200 pozos con los mayores valores
    top_wells = wells_data.sort_values(by='predictions', ascending=False).head(200)
    return top_wells

# Seleccionamos los 200 pozos con las mejores predicciones para cada región usando solo el conjunto de validación
top_wells_0 = top_200_wells(predictions_0, target_valid_0)
top_wells_1 = top_200_wells(predictions_1, target_valid_1)
top_wells_2 = top_200_wells(predictions_2, target_valid_2)

# mostramos los pozos seleccionados para cada region
print('mejores pozos para la región 0')
print(top_wells_0.head())
print()
print('mejores pozos para la región 1')
print(top_wells_1.head())
print()
print('mejores pozos para la región 2')
print(top_wells_2.head())




mejores pozos para la región 0
       predictions      actual
9317    180.180713  162.810993
219     176.252213  153.639837
10015   175.850623  162.153488
11584   175.658429   96.893581
23388   173.299686  178.879516

mejores pozos para la región 1
       predictions      actual
20430   139.818970  137.945408
7777    139.773423  137.945408
8755    139.703330  137.945408
1178    139.560938  137.945408
4285    139.516754  137.945408

mejores pozos para la región 2
       predictions      actual
22636   165.856833  175.103291
24690   165.679685  131.627481
7811    163.439962  141.160070
1581    162.062589  159.676082
6751    161.797476  142.135203


### volumen objetivo de reservas de los 200 pozos de cada región

In [97]:
# obtenemos la suma del volumen de reservas (predicho y real) para los 200 pozos seleccionados en cada región
def sum_reserves(top_wells):
    total_predicted = top_wells['predictions'].sum()
    total_actual = top_wells['actual'].sum()
    return total_predicted, total_actual

# Región 0
predicted_reserves_0, actual_reserves_0 = sum_reserves(top_wells_0)
print(f'Volumen total predicho en la región 0: {predicted_reserves_0:.2f}')
print(f'Volumen total real en la región 0: {actual_reserves_0:.2f}')
print()
# Región 1
predicted_reserves_1, actual_reserves_1 = sum_reserves(top_wells_1)
print(f'Volumen total predicho en la región 1: {predicted_reserves_1:.2f}')
print(f'Volumen total real en la región 1: {actual_reserves_1:.2f}')
print()
# Región 2
predicted_reserves_2, actual_reserves_2 = sum_reserves(top_wells_2)
print(f'Volumen total predicho en la región 2: {predicted_reserves_2:.2f}')
print(f'Volumen total real en la región 2: {actual_reserves_2:.2f}')



Volumen total predicho en la región 0: 31102.33
Volumen total real en la región 0: 29601.84

Volumen total predicho en la región 1: 27746.03
Volumen total real en la región 1: 27589.08

Volumen total predicho en la región 2: 29603.90
Volumen total real en la región 2: 28245.22


### Ganancia potencial de los 200 pozos principales por región

In [98]:
def calculate_profit(reserves_sum, wells=200, budget=100e6, income_per_unit=4500):
    # Calculamos el ingreso potencial
    total_income = reserves_sum * income_per_unit
    # Restamos el presupuesto para calcular la ganancia neta
    profit = total_income - budget
    return profit

# Calculamos el beneficio potencial para la región 0
profit_0 = calculate_profit(predicted_reserves_0)
print(f'Beneficio estimado para la región 0: {profit_0:.2f} dolares')

# Calculamos el beneficio potencial para la región 1
profit_1 = calculate_profit(predicted_reserves_1)
print(f'Beneficio estimado para la región 1: {profit_1:.2f} dolares')

# Calculamos el beneficio potencial para la región 2
profit_2 = calculate_profit(predicted_reserves_2)
print(f'Beneficio estimado para la región 2: {profit_2:.2f} dolares')


Beneficio estimado para la región 0: 39960488.77 dolares
Beneficio estimado para la región 1: 24857120.52 dolares
Beneficio estimado para la región 2: 33217543.96 dolares


La Región 0 es la que tiene el mayor beneficio estimado (39,960,488.77 dolares) de todas las regiones. Este beneficio es resultado de un volumen total de reservas predicho relativamente alto. Sin embargo, La diferencia entre el volumen predicho y el real no es muy grande (cerca de 1,500 miles de barriles). El modelo en esta región parece ser razonablemente preciso para hacer predicciones.

la Región 1 es la más precisa en términos de predicción, el beneficio (24,857,120.52 dolares) es considerablemente menor en comparación con las otras regiones. Esto se debe a que el volumen total de reservas predicho (y real) es más bajo que en las otras regiones. La diferencia entre el volumen predicho y el real es mínima (cerca de 150 miles de barriles). Esto indica que el modelo es muy preciso en la Región 1. Las predicciones están muy cerca de los valores reales.

En la Región 2, el modelo tiende a sobreestimar el volumen total de reservas en aproximadamente 1,300 miles de barriles. Aunque el error no es excesivo, es más alto en comparación con la Región 1 y menor que la region 0. ofrece un beneficio intermedio (33,217,543.96 dolares) en comparación con las Regiones 0 y 1. El volumen predicho es ligeramente sobreestimado, pero el beneficio sigue siendo mayor que el de la Región 1.

teniendo en cuenta lo anterior, la Región 0 es la más atractiva en términos de beneficio potencial. Aunque tiene una ligera sobreestimación en las reservas predichas pero la diferencia no es lo suficientemente grande como para representar un riesgo significativo y sigue ofreciendo el mejor margen de ganancias.

## Riesgos y ganancias para cada región utilizando bootstrapping

### Distribución de los beneficios

In [99]:
#Hacemos bootstrapping sobre los datos de pozos seleccionados para calcular la distribución de los beneficios.
def bootstrap_profit(data, n_samples=1000, wells=200, budget=100e6, income_per_unit=4500):
   
    # Creamos el generador de números aleatorios
    state = np.random.RandomState(12345)
    
    profits = []
    
    for i in range(n_samples):
        # Tomamos una muestra aleatoria (con reemplazo) de los pozos seleccionados
        sample = data.sample(n=wells, replace=True, random_state=state)

        # Calculamos el volumen total de reservas predicho para la muestra
        total_predicted_reserves = sample['predictions'].sum()

        # Calculamos el ingreso total generado para la muestra
        total_income = total_predicted_reserves * income_per_unit

        # Calculamos la ganancia neta restando el presupuesto
        profit = total_income - budget
        profits.append(profit)

    return profits

# Usamos las tablas top_wells_0, top_wells_1, top_wells_2 que ya tienen las predicciones y los valores reales
profits_0 = bootstrap_profit(top_wells_0)

profits_1 = bootstrap_profit(top_wells_1)

profits_2 = bootstrap_profit(top_wells_2)


### Cálculo del beneficio promedio, intervalo de confianza  y riesgo de pérdidas

In [100]:

def analyze_profits(profits, region):
    
    profits = np.array(profits)
    
    # hallamos el beneficio promedio
    average_profit = profits.mean()

    # calculamos el intervalo de confianza del 95%
    lower_ci = pd.Series(profits).quantile(0.025)
    upper_ci = pd.Series(profits).quantile(0.975)

    # calculamos el riesgo de pérdidas como porcentaje de ganancias negativas
    risk_of_loss = (profits < 0).mean() * 100

    # mostramos los resultados
    print(f"Beneficio Promedio de la ({region}): {average_profit:.2f}")
    print(f"Intervalo de Confianza 95% de la ({region}): [{lower_ci:.2f}, {upper_ci:.2f}]")
    print(f"Riesgo de Pérdidas de la ({region}): {risk_of_loss:.2f}%\n")

# Usamos la función para cada una de las regiones
analyze_profits(profits_0, region="región 0")
analyze_profits(profits_1, region="región 1")
analyze_profits(profits_2, region="región 2")


Beneficio Promedio de la (región 0): 39944044.81
Intervalo de Confianza 95% de la (región 0): [39104578.92, 40792451.06]
Riesgo de Pérdidas de la (región 0): 0.00%

Beneficio Promedio de la (región 1): 24856341.33
Intervalo de Confianza 95% de la (región 1): [24818499.87, 24894836.02]
Riesgo de Pérdidas de la (región 1): 0.00%

Beneficio Promedio de la (región 2): 33203436.83
Intervalo de Confianza 95% de la (región 2): [32503826.13, 33937189.01]
Riesgo de Pérdidas de la (región 2): 0.00%



### Conclusiones

La Región 0 tiene el beneficio promedio más alto (39,944,044.81 dolares), seguida de la Región 2 (33,203,436.83 dolares), mientras que la Región 1 tiene el beneficio más bajo (24,856,341.33 dolares). Esto indica que, en promedio, desarrollar pozos en la Región 0 daría mayores ingresos.

Los intervalos de confianza nos dice que la Región 0 sigue siendo la opción con el mayor margen de beneficio dentro del intervalo, lo que indicaria que los resultados son estables. La Región 1 tiene un intervalo muy ajustado, lo que indica que los beneficios no varían mucho, pero sigue siendo la región con menor rentabilidad. La Región 2 tiene un intervalo amplio, pero aún por debajo de la Región 0.

Todas las regiones tienen un riesgo de pérdidas de 0%, lo que significa que en ninguna de las simulaciones de bootstrapping se ve un escenario donde se incurra en pérdidas

por lo anterior se determinaria que La Región 0 seria la mejor opcion para el desarrollo, ya que tiene el beneficio promedio más alto (39,944,044.81 dolares), un intervalo de confianza que está por encima de los beneficios promedio de las otras dos regiones, y riesgo de pérdidas nulo. lo cual coincide con nuestra eleccion en el paso 4.3 donde se concluyó que la region 0 era la mas rentable.